# Bayesian Sharpe Ratio (Posterior Ranking for Asset Selection)

Chapter 10 models the Sharpe ratio as a probabilistic quantity. Here we use a
conjugate Normal-Inverse-Gamma model for daily returns to obtain a posterior
over $(\mu, \sigma^2)$ and then approximate the Sharpe distribution via Monte Carlo.

We rank assets by $P(\text{Sharpe} > 0)$ on each rebalance date.


# Chapter 10 Bayesian ML (Predictive): Bayesian Sharpe Ratio

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [1]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [2]:
from backtester.data import load_cleaned_assets, align_close_prices

assets_ohlcv = load_cleaned_assets(cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv).sort_index()
rets = close_prices.pct_change().dropna(how='all').fillna(0.0)

# Time split
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(rets.index).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))

# NIG hyperparameters (weakly informative)
mu0 = 0.0
kappa0 = 1.0
alpha0 = 2.0
beta0 = 1e-4

LOOKBACK = 252
S = 300
REBALANCE_FREQ = 'W'

# Build prediction matrix on the test window: score = P(Sharpe>0) - 0.5
test_idx = rets.loc[test_start:end].index
rebal_dates = pd.Series(test_idx, index=test_idx).resample(REBALANCE_FREQ).last().dropna().tolist()
score_rebal = pd.DataFrame(index=pd.DatetimeIndex(rebal_dates), columns=rets.columns, dtype=float)

def posterior_params(window: pd.DataFrame) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    x = window.to_numpy(dtype=float)
    valid = np.isfinite(x)
    n = valid.sum(axis=0).astype(float)
    x0 = np.where(valid, x, 0.0)
    xbar = np.divide(x0.sum(axis=0), np.maximum(1.0, n))
    centered = np.where(valid, x0 - xbar, 0.0)
    ss = (centered * centered).sum(axis=0)
    denom = np.maximum(1.0, n - 1.0)
    s2 = ss / denom
    kappa_n = kappa0 + n
    mu_n = (kappa0 * mu0 + n * xbar) / np.maximum(1e-12, kappa_n)
    alpha_n = alpha0 + 0.5 * n
    beta_n = beta0 + 0.5 * (n - 1.0) * s2 + 0.5 * (kappa0 * n / np.maximum(1e-12, kappa_n)) * (xbar - mu0) ** 2
    return mu_n, kappa_n, alpha_n, beta_n

for dt in score_rebal.index:
    end_pos = int(rets.index.searchsorted(dt, side='left'))
    start_pos = max(0, end_pos - LOOKBACK)
    window = rets.iloc[start_pos:end_pos]
    mu_n, kappa_n, alpha_n, beta_n = posterior_params(window)
    g = rng.gamma(shape=alpha_n[None, :], scale=1.0 / beta_n[None, :], size=(S, alpha_n.shape[0]))
    sigma2 = 1.0 / g
    mu = rng.normal(loc=mu_n[None, :], scale=np.sqrt(sigma2 / kappa_n[None, :]))
    sharpe = mu / np.sqrt(sigma2 + 1e-18)
    p_pos = (sharpe > 0.0).mean(axis=0)
    score_rebal.loc[dt] = p_pos - 0.5

pred_matrix = score_rebal.reindex(test_idx, method='ffill').fillna(0.0)


In [3]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Bayes Sharpe Ratio (P[SR>0])'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,33.782337,21.413185,1.315634,-13.072936
1,MPT,26.764329,17.129073,1.072677,-13.695820


,Bayes Sharpe Ratio (P[SR>0]) - 1N
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1337823.370352
Equity Peak,1337823.370352
Total Return [%],33.782337
CAGR [%],21.413185
Volatility (ann) [%],15.64573
Sharpe,1.315634


,Bayes Sharpe Ratio (P[SR>0]) - MPT
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1267643.288604
Equity Peak,1267643.288604
Total Return [%],26.764329
CAGR [%],17.129073
Volatility (ann) [%],15.887284
Sharpe,1.072677
